In [28]:
def add_leading_zeros(binary_number, expected_length):
	"""
	Adds leading zeros to a binary number so that the number of characters
	in the binary string matches the specified expected length and the value
	of the binary sring remains unchanged.

	Args:
		binary_number:    A string representation of a number in base 2
		expected_length:  Expected length of the binary number string

	Returns:
	    A binary string of a length specified by the argument with the 
	    same numerical value as the binary number from the first argument.
	"""
	length = len(binary_number)
	return (expected_length - length) * '0' + binary_number

def rgb_to_binary(r, g, b):
	"""
	Converts decimal numbers representing RGB values of a pixel into
	binary numbers of the same values.

	Args:
	    r:    Decimal representation of the red channel value
	    g:    Decimal representation of the green channel value
	    b:    Decimal representation of the blue channel value

	Returns:
	    Binary representations of the red, green, and blue channel values
	"""
	return add_leading_zeros(bin(r)[2:], 8), add_leading_zeros(bin(g)[2:], 8), add_leading_zeros(bin(b)[2:], 8)

In [29]:
from PIL import Image

def get_binary_pixel_values(img, width, height):
	"""
	Retrieves a string of concatenated binary representations of RGB channel values of all pixels in an image.

	Args:
	    img:    An RGB image
	    width:  Width of the image
	    height: Height of the image

	Returns:
	    A string with concatenated binary numbers representing the RGB channel values of all pixels in the image
	    where each binary number representing one channel value is 8 bits long, padded with leading zeros 
	    when necessary. Therefore, each pixel in the image is represented by 24 bit long binary sequence.
	"""
	hidden_image_pixels = ''
	for col in range(width):
		for row in range(height):
			pixel = img[col, row]
			r = pixel[0]
			g = pixel[1]
			b = pixel[2]
			r_binary, g_binary, b_binary = rgb_to_binary(r, g, b)
			hidden_image_pixels += r_binary + g_binary + b_binary
	return hidden_image_pixels

def change_binary_values(img_visible, hidden_image_pixels, width_visible, height_visible, width_hidden, height_hidden):
    """
    Modifies the binary values of a visible image to embed a hidden image.

    Args:
        img_visible: Pixel data of the visible image.
        hidden_image_pixels: Binary pixel data of the hidden image.
        width_visible: Width of the visible image.
        height_visible: Height of the visible image.
        width_hidden: Width of the hidden image.
        height_hidden: Height of the hidden image.

    Returns:
        Modified pixel data of the visible image.
    """
    idx = 0
    for col in range(width_visible):
        for row in range(height_visible):
            if col == 0 and row == 0:
                # Encode the hidden image dimensions in the top-left pixel
                w_h_binary = f"{width_hidden:012b}{height_hidden:012b}"
                img_visible[col, row] = (
                    int(w_h_binary[0:8], 2),
                    int(w_h_binary[8:16], 2),
                    int(w_h_binary[16:24], 2)
                )
                continue
            
            if idx + 24 <= len(hidden_image_pixels):
                r, g, b = img_visible[col, row]  # Ensure unpacking exactly 3 values
                r_binary, g_binary, b_binary = rgb_to_binary(r, g, b)
                r_binary = r_binary[0:4] + hidden_image_pixels[idx:idx+4]
                g_binary = g_binary[0:4] + hidden_image_pixels[idx+4:idx+8]
                b_binary = b_binary[0:4] + hidden_image_pixels[idx+8:idx+12]
                
                # Update the pixel with new binary values
                img_visible[col, row] = (
                    int(r_binary, 2),
                    int(g_binary, 2),
                    int(b_binary, 2)
                )
                idx += 12
            else:
                break  # Stop if there are no more hidden pixels to embed

    return img_visible


def encode(img_visible, img_hidden):
    """
    Encodes a hidden image into a visible image.
    
    Args:
        img_visible: The visible image (must be in RGB mode).
        img_hidden: The hidden image (must be in RGB mode).
    
    Returns:
        Encoded image with the hidden image embedded in the binary representation.
    """
    # Ensure images are in RGB mode
    if img_visible.mode != 'RGB':
        img_visible = img_visible.convert('RGB')
    if img_hidden.mode != 'RGB':
        img_hidden = img_hidden.convert('RGB')
    
    img_visible_copy = img_visible.load()
    img_hidden_copy = img_hidden.load()
    
    width_visible, height_visible = img_visible.size
    width_hidden, height_hidden = img_hidden.size
    
    # Retrieve binary pixel values from the hidden image
    hidden_image_pixels = get_binary_pixel_values(img_hidden_copy, width_hidden, height_hidden)
    
    # Modify binary values of the visible image to embed the hidden image
    encoded_image = change_binary_values(
        img_visible_copy, hidden_image_pixels,
        width_visible, height_visible, width_hidden, height_hidden
    )
    
    return img_visible  # Return the modified image



In [30]:
def extract_hidden_pixels(image, width_visible, height_visible, pixel_count):
	"""
	Extracts a sequence of bits representing a sequence of binary values of 
	all pixels of the hidden image.
	The information representing a hidden image is stored in the 4 least significant
	bits of a subset of pixels of the visible image.

	Args:
	    image:            An RGB image to recover a hidden image from
	    width_visible:    Width of the visible image
	    height_visible:   Height of the visible image
	    pixel_count:      Number of pixels in the hidden image

	Returns:
	    A binary string representing pixel values of the hidden image
	"""
	hidden_image_pixels = ''
	idx = 0
	for col in range(width_visible):
		for row in range(height_visible):
			if row == 0 and col == 0:
				continue
			r, g, b = image[col, row]
			r_binary, g_binary, b_binary = rgb_to_binary(r, g, b)
			hidden_image_pixels += r_binary[4:8] + g_binary[4:8] + b_binary[4:8]
			if idx >= pixel_count * 2:
				return hidden_image_pixels
	return hidden_image_pixels

def reconstruct_image(image_pixels, width, height):
    """
    Reconstructs the hidden image from binary pixel data.

    Args:
        image_pixels: A string of binary pixel data.
        width:        Width of the hidden image.
        height:       Height of the hidden image.

    Returns:
        A reconstructed RGB image.
    """
    from PIL import Image
    image = Image.new("RGB", (width, height))
    image_copy = image.load()
    
    idx = 0
    total_bits = len(image_pixels)
    for col in range(width):
        for row in range(height):
            # Ensure we have enough bits to extract a complete pixel
            if idx + 24 <= total_bits:
                r_binary = image_pixels[idx:idx+8]
                g_binary = image_pixels[idx+8:idx+16]
                b_binary = image_pixels[idx+16:idx+24]
                
                # Convert binary to integer values
                r = int(r_binary, 2)
                g = int(g_binary, 2)
                b = int(b_binary, 2)
                
                # Assign the RGB values to the image pixel
                image_copy[col, row] = (r, g, b)
                idx += 24
            else:
                # Fallback in case of incomplete data
                image_copy[col, row] = (0, 0, 0)  # Default to black pixel
                
    return image

	
def decode(image):
	"""
	Loads the image to recover a hidden image from, retrieves the information about the
	size of the hidden image stored in the top left pixel of the visible image,
	extracts the hidden binary pixel values from the image and reconstructs the hidden
	image.

	Args:
	    image:    An RGB image to recover a hidden image from

	Returns:
	    A recovered image, which was hidden in the binary representation of the visible image
	"""
	image_copy = image.load()
	width_visible, height_visible = image.size
	r, g, b = image_copy[0, 0]
	r_binary, g_binary, b_binary = rgb_to_binary(r, g, b)
	w_h_binary = r_binary + g_binary + b_binary
	width_hidden = int(w_h_binary[0:12], 2)
	height_hidden = int(w_h_binary[12:24], 2)
	pixel_count = width_hidden * height_hidden
	hidden_image_pixels = extract_hidden_pixels(image_copy, width_visible, height_visible, pixel_count)
	decoded_image = reconstruct_image(hidden_image_pixels, width_hidden, height_hidden)
	return decoded_image

In [33]:
import os
from PIL import Image

# Paths
img_visible_path = r"A:\work\image_Steganography\images\women_image.png"
img_hidden_path = r"A:\work\image_Steganography\images\senoir_image.jpg"
output_path = r"A:\work\image_Steganography\img\encoded_image.png"

# Ensure the output directory exists
output_dir = os.path.dirname(output_path)
os.makedirs(output_dir, exist_ok=True)

# Open images
img_visible = Image.open(img_visible_path)
img_hidden = Image.open(img_hidden_path)

# Encode the image
encoded_image = encode(img_visible, img_hidden)  # function call

# Save the encoded image
encoded_image.save(output_path)
print(f"Encoded image saved to {output_path}")


Encoded image saved to A:\work\image_Steganography\img\encoded_image.png


In [35]:
img_path = "img/encoded_image.png"
output_path = 'img/decoded_image.png'
decoded_image = decode(Image.open(img_path)) #funtion call
decoded_image.save(output_path)
